# CW2.2:  Compiler Back End for FUNC

Based on the AST from part 1, in this coursework you will develop the back-end that should generate MIPS code that can be emulated in the cpulator: 
https://cpulator.01xz.net/?sys=mipsr5b-spim

**CW2 Part II** is concerned with the implementation of the compiler’s back end. This is also worth 10 marks and released later. 

You are expected to use the labs slots, as well as private questions to Kathrin & the Lab Helpers throughout the course to work on it. 

**IMPORTANT** 
Compiler errors: All code you submit must compile. Programs that do not compile will receive an automatic zero.
- If you are having trouble getting your assignment to compile, please visit consulting hours.
- If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.

## Testing 

At the end of this file you'll find example program you can test your programs with. 
**You will want to write additional tests for intermediate steps.**

You will want to run your compiled program in an emulator: 

 https://cpulator.01xz.net/?sys=mipsr5b-spim

**The plagarism policy does not hold for this part of the coursework. 
Please feel free to share your tests with other students in the course.**

To simplify debugging assembler code, you can insert comments into your assembler code via the ``Verbatim`` constructor, e.g. 
```
[Verbatim "# Comment: Pushing on the stack."] @ push s8
```

## Submission

Please submit this notebook on Canvas until **Wed, 5th April**. 
Please ensure that you do not change the name or signature of the functions ``compile_exp``, ``compile_program``, etc. 

**Late Submissions.** See Canvas for F29LP's late-submission policy. 

**Plagarism.** All code (except tests) is subject to the course's plagarism policy. 

Happy coding!

## The Target Language 

We will again use the ``FUNC`` language from CW 2.1, see there for a description. 
We will start from its abstract representation. 

The goal is to write a backend that compiles this abstract representation into MIPS code. 
Like in the lectures, you can test your own code via the following emulator:
https://cpulator.01xz.net/?sys=mipsr5b-spim

Here are some requirements and assumptions you can make in addition to those already discussed in part 1: 
- You can assume that there are sufficient registers to hold all variables in a function.
- There is no global state.
- The code in the ``main()`` function should be executed (using the ``start_:`` label in MIPS). You have to put ``.global _start`` at the beginning of the assembly code to ensure that the assembler finds this label.
- All other functions can only be reached by calling them from ``main`` (possibly indirectly via other functions).
- Use the ``$s0-$s7`` registers to store variables of the ``main()`` function. 
- Use the ``$t0-$t7`` registers for the variables of other functions.
- Use ``$a0-$a3`` to store arguments. You can assume that there are not more than 4 arguments.
- Use ``$v0`` to store the return value. The variable declared in the optional return statement at the end of each method should be returned in $v0.
- Registers should be allocated in the order variables are introduced, starting with ``$s0/$t0``
- You can use ``$t8`` and ``$t9`` for intermediate computations as we have done for SIMP.
- ``main`` cannot be called by any functions (including ``main``).

Further requirements and assumptions are detailed for each relevant part below. 

Below you can see an abstract grammar for the language you've seen before.

In [ ]:
type exp = Numb of int | Id of string | App of string * exp list

type bop = Less | LessEq | Eq | NEq 
type cond = C of bop * exp * exp

type statement =
  Assign of string * exp
| Read of string 
| Write of exp 
| If of cond * statement list
| Ite of cond * statement list * statement list
| While of cond * statement list

type mmethod = M of string (* name of function *)
                * string list (* arguments *)
                * string list (* declarations *) 
                * statement list (* function body *)
                * string option (* possible return value value *)

type program = P of mmethod list

## Subtasks 


### Part 1: Basic Solution (4 Marks) 
The basic solution has to support compiling the ``main()`` function with all features except function calls. 
You should still support the built-in functions ``plus``, ``minus``, ``times``, and ``divide``.


### Part 2: Basic Function Calls (3 Marks)
Extend the basic solution with function calls. 
You do not need to support nested and recursive calls. 
However, you need to implement a suitable register-based protocol where ``$a0-$a3`` is used for arguments, and ``$v0`` for the result. You can assume a maximum of 4 arguments. Remember, you should use the ``$t0-$t7`` registers for local variables (except for ``main()`` where ``$s0-$s7`` are used). 

### Part 3: Nested and Recursive Functions (3 Marks)
Extend functions to allow recursive and nested calls. To achieve this, you need to develop a stack frame. As all variables are in registers, (when required) you will need to push a stack frame with all caller-save registers before the call is made and pop them back to the correct registers when control is returned to the caller.


# Definitions

In [ ]:
(* We represent registers as numbers. Registers are represented by 0 to 31. *)
type register = int

(* Value returned by a subroutine *)
let v0 : register = 2 
let v1 : register = 3 

(* Arguments to subroutine *)
let a0 : register = 4 
let a1 : register = 5
let a2 : register = 6
let a3 : register = 7

(* Temporary registers *)
let t0 : register = 8
let t1 : register = 9
let t2 : register = 10
let t3 : register = 11
let t4 : register = 12
let t5 : register = 13
let t6 : register = 14
let t7 : register = 15

(* Saved registers *)
let s0 : register = 16
let s1 : register = 17 
let s2 : register = 18 
let s3 : register = 19 
let s4 : register = 20 
let s5 : register = 21
let s6 : register = 22 
let s7 : register = 23 

(* Temporary registers $t8 and $t9 will be used for intermediate results. *)
let t8 : register = 24 (* $t8 *)
let t9 : register = 25 (* $t9 *)

let (sp : register) = 29 (* stack pointer *)
let (fp : register) = 30 (* frame pointer *)
let (ra : register) = 31 (* return address *)

(* We represent instructions as an abstract data type. *)

type label = string

type instruction =  Add of register * register * register (* add $1, $2, $3 - $1 = $2 + $3 *)
                   | Sub of register * register * register (* sub $1, $2, $3; $1 = $2 - $3 *)
                   | Addi of register * register * int (* addi $1, $2, 100 - $1 = $2 + 100, immediate means a constant number  *)
                   | Addiu of register * register * int (* addi $1, $2, 100 - $1 = $2 + 100, values treated as unsigned, immediate means a constant number  *)
                   | Mul of register * register * register (* mul $1, $2, $3 - $1 = $2 * $3, without overflow, result is only 32 bits *)
                   | Div of register * register (* div $2, $3 - $hi,$low=$2/$3, Remainder stored in special register hi, Quotient stored in special register lo   *)
                   | And of register * register * register (* and $1, $2, $3 - $1 = $2 & $3, bitwise AND *)
                   | Or of register * register * register (* or $1, $2, $3 - $1 = $2 | 100, bitwise OR *)
                   | Andi of register * register * int (* andi $1, $2, 100 - $1 = $2 & 100, bitwise AND with immediate value  *)
                   | Ori of register * register * int (* ori $1, $2, 100 - $1 = $2 | 100, bitwise OR with immediate value  *)
                   | Lw of register * int * register (* lw $1, 100 ($2) - load word, $1 = Memory[$2 + 100], copy from memory to register *)
                   | Sw of register * int * register (* sw $1, 100 ($2) - store word, Memory[$2 + 100] = $1, copy from register to memory *)
                   | La of register * label (* $1 = Address of label *) 
                   | Li of register * int (* li $1, 100 - Loads immediate value into register *)
                   | Move of register * register (* move $1,$2 - $1 = $2, Copy from register to register *)
                   | Mfhi of register (* mfhi $2, $2 = hi, copy from special register hi to general register *)
                   | Mflo of register (* mflo $2, $2 = lo, copy from special register lo to general register *)
                   | Label of label 
                   | Beq of register * register * string (* beq $1, $2, l - if ($1 == $2) go to label l *)
                   | Bne of register * register * string (* bne $1, $2, l - if ($1 != $2) go to label l *)
                   | Bgt of register * register * string (* bgt $1, $2, l - if ($1 > $2) go to label l *)
                   | Blt of register * register * string (* blt $1, $2, l - if ($1 < $2) go to label l *)
                   | Bge of register * register * string (* bge $1, $2, l - if ($1 >= $2) go to label l *)
                   | Ble of register * register * string (* ble $1, $2, l - if ($1 <= $2) go to label l *)                  
                   | J of label (* j l, go to label l, jumps to target address *)
                   | Jr of register (* jump register, jr $1, go to address stored in $1 *)
                   | Jal of label (* jump and link, e.g. jal l - $ra=PC+4; go to label l - used when making procedure call. This saves the return address in $ra.  *)
                   | SysCall 
                   | Verbatim of string (* Produce the given string verbatim in the assembly output *)
                   
type code = instruction list

let print_register (r : register) = 
    match r with 
    | 2 | 3 -> "$v" ^ (string_of_int (r - v0)) 
    | 4 |5 |6 | 7 -> "$a" ^ string_of_int (r - a0)
    | 8|9|10|11|12|13|14|15 -> "$t" ^ string_of_int (r - t0) 
    | 16|17|18|19|20|21|22|23 -> "$s" ^ string_of_int (r - s0)
    | 24 -> "$t8"
    | 25 -> "$t9"
    | 29 -> "$sp"
    | 30 -> "$fp"
    | 31 -> "$ra"
    | _ -> "$" ^ string_of_int r

let print_instruction (i : instruction) = match i with 
    | Add (r1, r2, r3) -> "add " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Sub (r1, r2, r3) -> "sub " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Addi (r1, r2, i) -> "addi " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ string_of_int i
    | Addiu (r1, r2, i) -> "addiu " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ string_of_int i
    | Mul (r1, r2, r3) -> "mul " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Div (r1, r2) -> "div " ^ print_register r1 ^ ", " ^ print_register r2
    | Beq (r1, r2, l) ->  "beq " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bne (r1, r2, l) ->  "bne " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bgt (r1, r2, l) ->  "bgt " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Blt (r1, r2, l) ->  "blt " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bge (r1, r2, l) ->  "bge " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Ble (r1, r2, l) ->  "ble " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Li (r, i) -> "li " ^ print_register r ^ ", " ^ string_of_int i
    | Lw (r1, o, r2) -> "lw " ^ print_register r1 ^ ", " ^ string_of_int o ^ "(" ^ print_register r2 ^ ")" 
    | La (r, l) -> "la " ^ print_register r ^ ", " ^  l
    | Sw (r1, o, r2) -> "sw " ^ print_register r1 ^ ", " ^ string_of_int o ^ "(" ^ print_register r2 ^ ")" 
    | Move (r1, r2) -> "move " ^ print_register r1 ^ ", " ^ print_register r2
    | Mfhi r -> "mfhi "^ print_register r
    | Mflo r -> "mflo "^ print_register r
    | And (r1, r2, r3) -> "and " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Andi (r1, r2, r3) -> "andi " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Or (r1, r2, r3) -> "or " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Ori (r1, r2, r3) -> "ori " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | SysCall -> "syscall"
    | Label l -> l ^ ":"
    | J label -> "j " ^ label 
    | Jr r -> "jr " ^ print_register r
    | Jal label -> "jal " ^ label
    | Verbatim s -> s
   
let rec print_code (c : code) : unit = match c with 
    | [] -> ()
    | c :: cs -> (print_endline (print_instruction c); print_code cs)
    
exception EEnv of string

let maxreg = 23

module Env = Map.Make (String)

(*  Function that finds the largest register in the environment. *)
let find_max_register (env : int Env.t) = 
    Env.fold (fun _ a b -> max a b) env (s0 - 1) 


(* Declaring a variable: 
  - When trying to declare a variable, and there are too many registers already 
    reserved, throw an exception. 
  - Else: Assign to variable x the largest register number + 1.
*)
let declare_var (env: int Env.t) (x:string) : int Env.t = 
    if (find_max_register env) >= maxreg
     then raise (EEnv "Too many variables")
     else Env.add x (1 + find_max_register env) env
     
exception E of string

(* Pushing the content of register r to the stack *)
let push (r : register) : code = [Addiu (sp, sp, -4);
                                  Sw (r, 0, sp)]

(* Popping the stack into register r *)
let pop (r : register) : code = [Lw (r, 0, sp);
                                 Addiu (sp, sp, 4)]
                                 
                                 
let counter : int ref = {contents = 0}

let next_val = 
    fun () ->
      counter := (!counter) + 1;
      !counter;;                                 

Here are some additional helper functions which might be useful:
- ``all_registers`` yields a list of all registers used by an environment. 
- ``declare_local_var`` behaves similar to ``declare_var``, but assigns the smallest unused **local register** (that is, ``$t0`` to ``$t7``) to a variable ``x``. 

In [ ]:
(* The following function yields a list of all registers used by an environment env *)
let all_registers env = Env.fold (fun k x xs -> x :: xs) env []

(* Variants of declare_var for local variables. *)
let maxlocalreg = 15

(*  Function that finds the largest local register in the environment. *)
let find_max_local_register (env : int Env.t) = 
    Env.fold (fun _ a b -> if a > maxlocalreg then b else max a b) env (t0 - 1) 

(* Declaring a local variable: 
  - When trying to declare a local variable, and there are too many registers already 
    reserved, throw an exception. 
  - Else: Assign to variable x the largest local register number + 1.
*)
let declare_local_var (env: int Env.t) (x:string) : int Env.t = 
    if (find_max_local_register env) >= maxlocalreg
     then raise (EEnv "Too many variables")
     else Env.add x (1 + find_max_local_register env) env

## Your Code

Fill out the gaps below.

In [ ]:
(* Compiling code that saves the value of an expression e into register r. 
   The basic solution can ignore the case e = App (f, es) except for 
   f = "plus", f = "minus", f = "times", and f = "divide".
   
   The basic function calls solution can ignore nested and recursive calls.
*)
let rec compile_exp env (r : register) (e : exp)  : code = match e with 
    | _ -> raise (E "To implement")

(* Generate code that saves the values of a list of expressions into a list of registers.
   This function will probably only be called if you implement function calls.  *)
and compile_exps env (rs : register list) (es : exp list) : code = match rs, es with 
    | _, _ -> raise (E "To implement")

(* Generate code for a single command. *)
let rec compile_cmd env (c : statement) : code = match c with 
    | _ -> raise (E "To implement")

(* Generate code for a list of commands. *)
and compile_cmds env (cs : statement list) : code = match cs with 
    | _ -> raise (E "To implement")

In [ ]:
(* Generate code for a method.
   For basic code generation only the first case has to be filled out.
*)
let rec compile_method (m : mmethod) : code = match m with 
 | M ("main", args, dcls, stms, ret) -> 
    raise (E "To implement")
 | M (name, args, dcls, stms, ret) -> 
    raise (E "To implement")


(* Generate code for a list of methods. *)
let rec compile_methods (ms : mmethod list) : code = match ms with 
    | [] -> [] 
    | m :: ms -> compile_method m @ compile_methods ms

(* Generate code for a whole program. *)
let compile_program (p : program) = match p with 
    | P ms -> raise (E "To implement")

## Example Programs 

Below are the three example programs from the appendix as an AST. 
You can see example MIPS output for the full solution in the appendix.

In [ ]:
let parsed_basic = 
  P
    [M ("main", [], ["inp"; "res"],
      [Read "inp"; Assign ("res", Numb 0);
       While (C (Less, Numb 0, Id "inp"),
        [Assign ("res", App ("plus", [Id "res"; Id "inp"]));
         Assign ("inp", App ("minus", [Id "inp"; Numb 1]))]);
       Write (Id "res")],
      None)];;

print_code (compile_program parsed_basic)

In [ ]:
let parsed_basic_function = 
P
    [M ("sum", ["inp"], ["res"],
      [Assign ("res", Numb 0);
       While (C (Less, Numb 0, Id "inp"),
        [Assign ("res", App ("plus", [Id "res"; Id "inp"]));
         Assign ("inp", App ("minus", [Id "inp"; Numb 1]))])],
      Some "res");
     M ("main", [], ["inp"; "res"],
      [Read "inp"; Assign ("res", App ("sum", [Id "inp"])); Write (Id "res")],
      None)];;

print_code (compile_program parsed_basic_function)

In [ ]:
let parsed_nested = 
P
    [M ("sum", ["inp"], ["tmp"; "res"],
      [Ite (C (Eq, Id "inp", Numb 0), [Assign ("res", Id "inp")],
        [Assign ("tmp", App ("sum", [App ("minus", [Id "inp"; Numb 1])]));
         Assign ("res", App ("plus", [Id "tmp"; Id "inp"]))])],
      Some "res");
     M ("main", [], ["inp"; "res"],
      [Read "inp"; Assign ("res", App ("sum", [Id "inp"])); Write (Id "res")],
      None)];;

print_code (compile_program parsed_nested)

## Appendix: Examples of FUNC code & generated MIPS code

### Basic Solution 

The following code should be supported in the basic solution:
```
method main() vars inp, res
begin
    read inp;
    res:=0;
    while less(0,inp)
    begin
        res := plus(res,inp);
        inp := minus(inp,1); 
    endwhile;
    write res;
endmethod;
```

The following MIPS code is a possible solution: 

```
.global _start
.set noreorder
.data
sinp:
.asciiz "INPUT>"  
.text
_start:
li $v0, 4
addiu $sp, $sp, -4
sw $a0, 0($sp)
la $a0, sinp
syscall
li $v0, 5
syscall
move $s0, $v0
lw $a0, 0($sp)
addiu $sp, $sp, 4
li $s1, 0
WLOOP4:
li $t8, 0
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $s0
lw $t8, 0($sp)
addiu $sp, $sp, 4
bge $t8, $t9, WEND4
move $t8, $s1
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $s0
lw $t8, 0($sp)
addiu $sp, $sp, 4
add $s1, $t8, $t9
move $t8, $s0
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 1
lw $t8, 0($sp)
addiu $sp, $sp, 4
sub $s0, $t8, $t9
j WLOOP4
WEND4:
li $v0, 1
addiu $sp, $sp, -4
sw $a0, 0($sp)
move $a0, $s1
syscall
lw $a0, 0($sp)
addiu $sp, $sp, 4
li $v0, 10
syscall
```

It should sum up all numbers from 0 to the given number. For example, if it reads ``3`` then it should print ``6`` (computing ``3+2+1``). Note that your solution does not have to be identical as long as the program produces the same answer and satisfies the specification in this document. 

## Basic Function Calls

The following program illustrates an example that should work with basic function calls. It does the same as the previous one, but with the computation that sums up the result in a separate function: 

```
method sum(inp) vars res
    begin
    res:=0;
    while less(0,inp)
    begin
        res := plus(res,inp);
        inp := minus(inp,1); 
    endwhile;
    return res;
endmethod;

method main() vars inp,res
    begin
    read inp;
    res := sum(inp);
    write res;
endmethod;
```

The following MIPS code is a correct generation: 

```
.global _start
.set noreorder
.data
sinp:
.asciiz "INPUT>"  
.text
sum:
li $t0, 0
WLOOP5:
li $t8, 0
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $a0
lw $t8, 0($sp)
addiu $sp, $sp, 4
bge $t8, $t9, WEND5
move $t8, $t0
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $a0
lw $t8, 0($sp)
addiu $sp, $sp, 4
add $t0, $t8, $t9
move $t8, $a0
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 1
lw $t8, 0($sp)
addiu $sp, $sp, 4
sub $a0, $t8, $t9
j WLOOP5
WEND5:
move $v0, $t0
jr $ra
_start:
li $v0, 4
addiu $sp, $sp, -4
sw $a0, 0($sp)
la $a0, sinp
syscall
li $v0, 5
syscall
move $s0, $v0
lw $a0, 0($sp)
addiu $sp, $sp, 4
# Start pushing stack frame
addiu $sp, $sp, -4
sw $ra, 0($sp)
# End pushing stack frame
move $a0, $s0
jal sum
# Start popping stack frame
lw $ra, 0($sp)
addiu $sp, $sp, 4
# End popping stack frame
move $s1, $v0
li $v0, 1
addiu $sp, $sp, -4
sw $a0, 0($sp)
move $a0, $s1
syscall
lw $a0, 0($sp)
addiu $sp, $sp, 4
li $v0, 10
syscall
```

Again, your solution does not have to be identical as long as the program produces the same answer and satisfies the specification in this document. 


## Nested/Recursive Calls

The following program illustrates support for nested and function calls. It does the same as the previous one, but computes the result using recursion:

```
method sum(inp) vars tmp, res
begin
    if eq(inp,0) then
        res := inp;
    else
        tmp := sum(minus(inp,1));
        res := plus(tmp,inp);
    endif;
    return res;
endmethod;

method main() vars inp,res
begin
    read inp;
    res := sum(inp);
    write res;
endmethod;
```

The following MIPS code is a possible solution: 
```
.global _start
.set noreorder
.data
sinp:
.asciiz "INPUT>"  
.text
sum:
move $t8, $a0
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 0
lw $t8, 0($sp)
addiu $sp, $sp, 4
bne $t8, $t9, IFFALSE3
move $t1, $a0
j IFEND3
IFFALSE3:
# Start pushing stack frame
addiu $sp, $sp, -4
sw $ra, 0($sp)
addiu $sp, $sp, -4
sw $t0, 0($sp)
addiu $sp, $sp, -4
sw $t1, 0($sp)
addiu $sp, $sp, -4
sw $a0, 0($sp)
# End pushing stack frame
move $t8, $a0
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 1
lw $t8, 0($sp)
addiu $sp, $sp, 4
sub $a0, $t8, $t9
jal sum
# Start popping stack frame
lw $a0, 0($sp)
addiu $sp, $sp, 4
lw $t1, 0($sp)
addiu $sp, $sp, 4
lw $t0, 0($sp)
addiu $sp, $sp, 4
lw $ra, 0($sp)
addiu $sp, $sp, 4
# End popping stack frame
move $t0, $v0
move $t8, $t0
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $a0
lw $t8, 0($sp)
addiu $sp, $sp, 4
add $t1, $t8, $t9
IFEND3:
move $v0, $t1
jr $ra
_start:
li $v0, 4
addiu $sp, $sp, -4
sw $a0, 0($sp)
la $a0, sinp
syscall
li $v0, 5
syscall
move $s0, $v0
lw $a0, 0($sp)
addiu $sp, $sp, 4
# Start pushing stack frame
addiu $sp, $sp, -4
sw $ra, 0($sp)
# End pushing stack frame
move $a0, $s0
jal sum
# Start popping stack frame
lw $ra, 0($sp)
addiu $sp, $sp, 4
# End popping stack frame
move $s1, $v0
li $v0, 1
addiu $sp, $sp, -4
sw $a0, 0($sp)
move $a0, $s1
syscall
lw $a0, 0($sp)
addiu $sp, $sp, 4
li $v0, 10
syscall
```

Again, your solution does not have to be identical as long as the program produces the same answer and satisfies the specification in this document. 